# LAB7 Assignment
> The document description are designed by JIa Yanhong in 2022. Oct. 20th
------

## LAB Assignment
### Exercise 1 logistic regression (20 points )
This exercise uses dataset digit01.csv , which has 13 columns, and the last column is the dependent variable. 

This part requires you to implement a `logistic regression` using the pytorch framework (defining a logistic regression class that inherits `nn.module`). To test your model, we provide a dataset `digit01.csv` which is in the **datasets folder**. This dataset requires you to divide the training set and the test set by yourself, and it is recommended that 80% of the training set and 20% of the test set be used.

+ load datasets

In [167]:
########### Write Your Code Here ###########
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore") 

df = pd.read_csv("datasets/digit01.csv", header=None)
df.head()

y = df[12]
X = df.drop(12, axis=1)
X.shape
############################################

(64, 12)

+ Splitting dataset into 80% Training and 20% Testing Data:

In [168]:
########### Write Your Code Here ###########

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, shuffle=True)
############################################


+ Define a LogisticRegression subclass of nn. Module

In [169]:
# Define a LogisticRegression subclass of nn. Module.
########### Write Your Code Here ###########
import  torch 

X_train = torch.from_numpy(X_train.values).float()
X_test = torch.from_numpy(X_test.values).float()
y_train = torch.from_numpy(y_train.values).long()
y_test = torch.from_numpy(y_test.values).long()
############################################





+ Create the model

In [170]:

########### Write Your Code Here ###########
import torch.nn.functional as F
from torch import nn

class LogisticRegression (nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.net = nn.Sequential(nn.Linear(12, 2))
    def forward(self, input):
        x = input.view([-1, 12])
        x = self.net(x)
        
        return F.log_softmax(x, dim= -1)
############################################

model_logisticRegession = LogisticRegression()

 + Loss function

In [171]:
########### Write Your Code Here ###########
def cross_entropy_loss(output, y):
    return F.nll_loss(output, y)
############################################

+ The optimizer

In [172]:
########### Write Your Code Here ###########
from torch.optim import SGD
optimizer_SGD = SGD(model_logisticRegession.parameters(), lr=0.001, momentum=0.9,
                         dampening=0.5, weight_decay=0.01, nesterov=False)
############################################

+ training Model

In [173]:
########### Write Your Code Here ###########
use_cuda = torch.cuda.is_available()


def train_logistic(epoch, model:nn.Module, optimizer, X_train, y_train) :
    model.train()
    for _ in range(epoch):
        
        if use_cuda:
            model = model.cuda()
            X_train, y_train = X_train.cuda(), y_train.cuda()

            output = model(X_train)
            loss = cross_entropy_loss(output, y_train)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if _ % 100 == 0:
                #save model
            
                print('Epoch: %d, loss : %lf' %(_, loss.item()))

EPOCH = 10000
train_logistic(EPOCH, model=model_logisticRegession, optimizer= optimizer_SGD, X_train = X_train, y_train = y_train)
############################################

Epoch: 0, loss : 0.751581
Epoch: 100, loss : 0.485726
Epoch: 200, loss : 0.382759
Epoch: 300, loss : 0.320889
Epoch: 400, loss : 0.278021
Epoch: 500, loss : 0.246609
Epoch: 600, loss : 0.222672
Epoch: 700, loss : 0.203859
Epoch: 800, loss : 0.188693
Epoch: 900, loss : 0.176207
Epoch: 1000, loss : 0.165742
Epoch: 1100, loss : 0.156840
Epoch: 1200, loss : 0.149168
Epoch: 1300, loss : 0.142481
Epoch: 1400, loss : 0.136595
Epoch: 1500, loss : 0.131369
Epoch: 1600, loss : 0.126695
Epoch: 1700, loss : 0.122484
Epoch: 1800, loss : 0.118668
Epoch: 1900, loss : 0.115191
Epoch: 2000, loss : 0.112006
Epoch: 2100, loss : 0.109077
Epoch: 2200, loss : 0.106372
Epoch: 2300, loss : 0.103864
Epoch: 2400, loss : 0.101531
Epoch: 2500, loss : 0.099355
Epoch: 2600, loss : 0.097318
Epoch: 2700, loss : 0.095407
Epoch: 2800, loss : 0.093610
Epoch: 2900, loss : 0.091916
Epoch: 3000, loss : 0.090315
Epoch: 3100, loss : 0.088801
Epoch: 3200, loss : 0.087365
Epoch: 3300, loss : 0.086002
Epoch: 3400, loss : 0.0847


+ Model Performance


In [174]:
########### Write Your Code Here ###########
import numpy as np
def test(model:nn.Module, X_test, y_test):
    model.eval()
    ACC_list = []
    loss_list = []
    pred_list = []
    
    with torch.no_grad():
        if use_cuda:
            model = model.cuda()
            X_test, y_test = X_test.cuda(), y_test.cuda()
        output = model(X_test)
        loss = cross_entropy_loss(output, y_test)

        pred = output.argmax(dim=1, keepdim=True)
        ACC = pred.eq(y_test.view_as(pred)).float().mean()
            
        ACC_list.append(ACC.cpu())
        loss_list.append(loss.cpu())
        pred_list.extend(pred.cpu().flatten())

    print('average ACC: ',np.mean(ACC_list))
    print('average loss: ',np.mean(loss_list))
    return pred_list
test(model=model_logisticRegession, X_test=X_test,y_test=y_test)
############################################

average ACC:  1.0
average loss:  0.04008827


[tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0)]

In [175]:
#classification report
from sklearn.metrics import classification_report
y_predicted_cls = test(model=model_logisticRegession, X_test=X_test,y_test=y_test)
print(classification_report(y_test, y_predicted_cls))

average ACC:  1.0
average loss:  0.04008827
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         7

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13



### Exercise 2  Handwriting recognition with MLP

Like last week's lab , your task in this section is also about recognizing handwritten digits, but you are required to use MLP to complete the exercise. It is recommended that you define an MLP class, which is a subclass of `nn.module`.

<font color='red' size=4>Note that your accuracy in this section will directly determine your score.</font>

For this exercise we use the `minist` dataset.

+ load datasets

In [183]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Normalize, Compose
device = torch.device("cuda:0") 
use_cuda = torch.cuda.is_available()

batch_size=60000
transform=Compose([ToTensor()])
train_dataset=MNIST(root='datasets/',train=True,download=False,transform=transform)
train_loader=DataLoader(train_dataset,shuffle=True,batch_size=batch_size)

test_dataset=MNIST(root='datasets/',train=False,download=False,transform=transform)
test_loader=DataLoader(test_dataset,shuffle=False,batch_size=batch_size)

for batch_idx, data in enumerate(train_loader, 0):
    inputs, targets = data 
    x=inputs.view(-1,28*28) 
    x_std=x.std().item() 
    x_mean=x.mean().item() 

print('mean:'+str(x_mean))
print('std:'+str(x_std))

mean:0.13066047430038452
std:0.30810782313346863


In [184]:
########### Write Your Code Here ###########
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Normalize, Compose

BATCH_SIZE = 128

transform = Compose(
[ToTensor(), Normalize(mean=(0.1307), std=(0.3081)).to(device=device)]    
)



batch_size=60000
train_dataset = MNIST(root='datasets/',train=True,download=False, transform=transform)
train_loader=DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)

test_dataset=MNIST(root='datasets/',train=False, download=False, transform=transform)
test_loader=DataLoader(test_dataset,shuffle=False,batch_size=BATCH_SIZE)


############################################                      

+ Define a MLP subclass of nn. Module

In [185]:
########### Write Your Code Here ###########
import torch.nn.functional as F
from torch import nn
#input size [BATCH_SIZE, 28, 28]

class MnistModel (nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(28 * 28,200),nn.ReLU())
        self.layer2 = nn.Sequential(nn.Linear(200,100),nn.ReLU())
        self.layer3 = nn.Sequential(nn.Linear(100,20),nn.ReLU())
        self.layer4 = nn.Linear(20,10)

        self.net = nn.Sequential(self.layer1, self.layer2, self.layer3, self.layer4)

    def forward(self, input):
        if use_cuda:
            input = input.cuda()
        x = input.view([-1, 28 * 28])
        x = self.net(x)
        
        return F.log_softmax(x, dim= -1)
############################################




+ Create the model

In [190]:
########### Write Your Code Here ###########
import torch
import os
model = MnistModel().to(device)
if os.path.exists('./model/model.pkl'):
    model.load_state_dict(state_dict= torch.load('./model/model.pkl'))
############################################

 + Loss function

In [187]:
########### Write Your Code Here ###########
def cross_entropy_loss(output, y):
    return F.nll_loss(output, y)
############################################

+ The optimizer

In [192]:
########### Write Your Code Here ###########
from torch.optim import Adam
optimizer_Adam = Adam(model.parameters(), lr=0.0001)
if os.path.exists('./model/optimizer.pkl'):
    optimizer_Adam.load_state_dict(state_dict= torch.load('./model/optimizer.pkl'))
############################################

+ training Model

In [193]:

########### Write Your Code Here ###########




def train(epoch, model:nn.Module, optimizer, data_loader) :
    model.train()
    for _ in range(epoch):
        for idx, (input, target) in enumerate(data_loader):
            if use_cuda:
                model = model.cuda()
                input, target = input.cuda(), target.cuda()

            output = model(input)
            loss = cross_entropy_loss(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if idx % 100 == 0:
                #save model
                torch.save(model.state_dict(), './model/model.pkl')
                torch.save(optimizer.state_dict(), './model/optimizer.pkl')


                print('Epoch: %d, idx: %d, loss : %lf' %(_, idx, loss.item()))

EPOCH = 50
train(EPOCH, model=model, optimizer= optimizer_Adam, data_loader = train_loader)
############################################

Epoch: 0, idx: 0, loss : 0.000008
Epoch: 0, idx: 100, loss : 0.000004
Epoch: 0, idx: 200, loss : 0.000006
Epoch: 0, idx: 300, loss : 0.000003
Epoch: 0, idx: 400, loss : 0.000002
Epoch: 1, idx: 0, loss : 0.000005
Epoch: 1, idx: 100, loss : 0.000001
Epoch: 1, idx: 200, loss : 0.000001
Epoch: 1, idx: 300, loss : 0.000002
Epoch: 1, idx: 400, loss : 0.000001
Epoch: 2, idx: 0, loss : 0.000001
Epoch: 2, idx: 100, loss : 0.000003
Epoch: 2, idx: 200, loss : 0.000003
Epoch: 2, idx: 300, loss : 0.000001
Epoch: 2, idx: 400, loss : 0.000002
Epoch: 3, idx: 0, loss : 0.000006
Epoch: 3, idx: 100, loss : 0.000005
Epoch: 3, idx: 200, loss : 0.000002
Epoch: 3, idx: 300, loss : 0.000007
Epoch: 3, idx: 400, loss : 0.000002
Epoch: 4, idx: 0, loss : 0.000004
Epoch: 4, idx: 100, loss : 0.000001
Epoch: 4, idx: 200, loss : 0.000002
Epoch: 4, idx: 300, loss : 0.000006
Epoch: 4, idx: 400, loss : 0.000003
Epoch: 5, idx: 0, loss : 0.000004
Epoch: 5, idx: 100, loss : 0.000002
Epoch: 5, idx: 200, loss : 0.000001
Epoc

+ Model Performance

In [194]:
########### Write Your Code Here ###########
import numpy as np
def test(model:nn.Module, data_loader):
    model.eval()
    ACC_list = []
    loss_list = []
    pred_list = []
    for idx, (input, target) in enumerate(data_loader):
        with torch.no_grad():
            if use_cuda:
                model = model.cuda()
                input, target = input.cuda(), target.cuda()
            output = model(input)
            loss = cross_entropy_loss(output, target)

            pred = output.argmax(dim=1, keepdim=True)
            ACC = pred.eq(target.view_as(pred)).float().mean()
            
            ACC_list.append(ACC.cpu())
            loss_list.append(loss.cpu())
            pred_list.extend(pred.cpu().flatten())

    print('average ACC: ',np.mean(ACC_list))
    print('average loss: ',np.mean(loss_list))
    return pred_list
test(model=model, data_loader=test_loader)
############################################

average ACC:  0.97992486
average loss:  0.1724151


[tensor(7),
 tensor(2),
 tensor(1),
 tensor(0),
 tensor(4),
 tensor(1),
 tensor(4),
 tensor(9),
 tensor(5),
 tensor(9),
 tensor(0),
 tensor(6),
 tensor(9),
 tensor(0),
 tensor(1),
 tensor(5),
 tensor(9),
 tensor(7),
 tensor(3),
 tensor(4),
 tensor(9),
 tensor(6),
 tensor(6),
 tensor(5),
 tensor(4),
 tensor(0),
 tensor(7),
 tensor(4),
 tensor(0),
 tensor(1),
 tensor(3),
 tensor(1),
 tensor(3),
 tensor(4),
 tensor(7),
 tensor(2),
 tensor(7),
 tensor(1),
 tensor(2),
 tensor(1),
 tensor(1),
 tensor(7),
 tensor(4),
 tensor(2),
 tensor(3),
 tensor(5),
 tensor(1),
 tensor(2),
 tensor(4),
 tensor(4),
 tensor(6),
 tensor(3),
 tensor(5),
 tensor(5),
 tensor(6),
 tensor(0),
 tensor(4),
 tensor(1),
 tensor(9),
 tensor(5),
 tensor(7),
 tensor(8),
 tensor(9),
 tensor(3),
 tensor(7),
 tensor(4),
 tensor(6),
 tensor(4),
 tensor(3),
 tensor(0),
 tensor(7),
 tensor(0),
 tensor(2),
 tensor(9),
 tensor(1),
 tensor(7),
 tensor(3),
 tensor(2),
 tensor(9),
 tensor(7),
 tensor(7),
 tensor(6),
 tensor(2),
 ten

In [195]:
#classification report

from sklearn.metrics import classification_report




y_predicted_cls = test(model=model, data_loader=test_loader)
print(len(y_predicted_cls))
y_test = test_dataset.targets


print(classification_report(y_test, y_predicted_cls))

average ACC:  0.97992486
average loss:  0.1724151
10000
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.98      0.98      0.98      1010
           4       0.98      0.98      0.98       982
           5       0.98      0.98      0.98       892
           6       0.98      0.98      0.98       958
           7       0.98      0.97      0.98      1028
           8       0.97      0.98      0.98       974
           9       0.97      0.97      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



### Exercise 3  Questions (10 points )

#### 1.What's the difference between logistic regression and Perceptron?

1.逻辑回归的loss function 是训练集上的极大似然函数，感知机是误分数据点到分割超平面的距离的和。优化目标不一样，前者是寻找参数最大化似然。后者是寻找参数最小化误分点到分割平面的距离。虽然都是用梯度下降。

2.逻辑回归本质上是回归，回归是需要输出一个连续的数值，所以它寻求的是the probability of positive output given an input.

#### 2.Advantages and disadvantages of neural networks?

优点：

1.具有自学习功能。例如实现图像识别时，只在先把许多不同的图像样板和对应的应识别的结果输入人工神经网络，网络就会通过自学习功能，慢慢学会识别类似的图像。

2.具有高速寻找优化解的能力。

缺点：

1.“黑盒”，具有不可解释性

2.数据量低时，表现较差

#### 3.What is the role of Activation Function in Neural networks?

激活函数的作用是去线性化，解决线性模型的表达、分类能力不足的问题。改变之前数据的线性关系，如果网络中全部是线性变换，则多层网络可以通过矩阵变换，直接转换成一层神经网络。若没有激活函数，神经网络只是线性模型的线性组合